# Converting a Normal EPANET .inp File to a CV-Res IWS File [1]
This notebook takes an input EPANET files with demands input normally as a CWS base demand and adds the necessary elements to convert it into a CV-Res IWS simulation  
For details on the CV-Res method refer to [1]  
A simplified schematic of the modified demand node in STM is seen below:  
  
  
![](CV-Res.png)

### First, we import the necessary libraries and packages
**WNTR** for building EPANET network models in Python  
**NUMPY & PANDAS** for data handling and processing  
**re** for searching and matching text in the .inp file using regular expressions

In [31]:
import wntr  
import numpy as np 
import pandas as pd
import re
import pathlib

### Specifying paths for simulation files and preprocessing the input
**Warning:** *Paths in this script (and the rest of this repo) are absolute unless running the network files provided within the repo*  
Input filename (with extensions) as string.  
For running the .inp files in this repository, you can use this relative path `"../Network-Files/Network X/"` where X is the network number 

In [32]:
 # Replace with appropriate path and filename
directory=pathlib.Path("../Network-Files/Network 1")
filename=pathlib.Path("Network1_12hr_PDA.inp")
name_only=str(filename.stem)[0:-4]
print("Selected File: ",name_only)
path=directory/filename

Selected File:  Gupta_and_Bhave_12hr


### Necessary Assumptions Input
Converting a CWS demand-driven analysis into an IWS pressure-driven analysis requires some assumptions in all methods  
The resistance of the service connection between the demand node and the household (end-user) is uncertain and is modelled using two assumptions  
The **desired head (pressure)** is the pressure at the demand node at which (or above) the consumer can satisfy their full demand in the supply duration (or possible less)  
The **minimum head (pressure)** is the minimum pressure at the demand node required for flow to begin passing through the service connection  
These two assumptions dictate the flow-pressure relationship that determines the pressure-dependent flow through the service connection as follows:

$$ Q\, = \!Q_{des}\sqrt{\frac{H_{j}-H^{min}}{H^{des}-H^{min}}} \quad[2]$$ 
Where Q is the flow through the service connection, $Q_{des}$ is the desired (base) demand, $H_j$ is the head at the demand node $j$, $H^{min}$ is the minimum head, and $H^{des}$ is the desired head


In [33]:
desired_pressure=10     # Set the desired pressure
minimum_pressure=0      # Set the minimum pressure
pressure_diff=desired_pressure-minimum_pressure  

### Extracting information from the input file
To modify the .inp file, demand node IDs, elevations, x and y coordinates  
We use wntr to build the network model of the input file and use wntr's junctions module to extract the details of each node

In [34]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]         # For storing elevations of demand nodes
xcoordinates=[]       # For storing x coordinates of demand nodes
ycoordinates=[]       # For storing y coordinates of demand nodes
all_nodes=[]          # For storing list of node ids of all nodes
all_elevations=[]     # For storing elevations of all nodes
## MAYBE SAVE ALL NODE IDS IN DATAFRAME WITH ELEVATION AND BASE DEMAND AND THEN FILTER DATA FRAME LATER FOR DEMAND NODES ONLY

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    all_nodes.append(node[1].name)
    all_elevations.append(node[1].elevation)
    # For all nodes that have non-zero demands
    if node[1].base_demand != 0:
        # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
        demand_nodes.append(node[1].name)
        desired_demands.append(node[1].base_demand)
        elevations.append(node[1].elevation)
        xcoordinates.append(node[1].coordinates[0])
        ycoordinates.append(node[1].coordinates[1])



# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

### Writing Reservoirs Section
For every demand node, an artificial reservoir (AR) will be added at an elevation of $E_{reservoir}=E_{Demand\,Node}+H^{min}$  
Raising the elevation by $H^{min}$ enforces the minimum pressure threshold as no flow to the user would occur if the pressure at the node is less than that value  

In [35]:
# Adds "AR" to each demand node id to be used as ID for AR
reservoirids=["AR"+str(id) for id in demand_nodes]
# Calculates the elevation of the AR
reservoir_elevs=[elevation + minimum_pressure for elevation in elevations ]
# No Patterns are assigned to any of the ARs
reservoir_patterns=["    "]*len(reservoirids)
# Semicolons to end each line
semicolons=[";"]*len(reservoirids)
# Dataframe with all the required fields for AR [ID   Elevation   Pattern   ;]
added_reservoirs=pd.DataFrame(list(zip(reservoirids,reservoir_elevs,reservoir_patterns,semicolons)))
# Exports the added reservoirs as a list of strings where each entry is a line of the reservoirs section
added_reservoirs=added_reservoirs.to_string(header=False,index=False).splitlines()

### Writing Added Pipes
Similar to the tanks section, each field for the pipe entries is stored in a separate list  
The added pipes connect the created tanks to the original demand nodes and are responsible for simulating the pressure-dependent flow relationship  
The length of the pipe is calculated according to the required head flow relationship dictated by the minimum and desired pressure as:  
$$L=(H^{des}-H^{min})\frac{C^{1.852}D^{4.87}}{10.67Q_{des}^{1.852}}$$  
The diameter is unified at 0.05 m (50 mm)

In [36]:
# Adds the phrase PipeforNode to each node id and stores it as a pipe id
pipeids=['PipeforNode'+str(id) for id in demand_nodes]
# Calculates the length of each pipe to simulate the head-flow relationship
lengths=[round(pressure_diff*130**1.852*0.05**4.87/10.67/(demand)**1.852,4) for demand in desired_demands]
# Sets all diameters to 1 m (1000 mm)
diameters_pipes=[50]*len(pipeids)
# Sets all Hazen-Williams Coefficients as 130
hazen=[130]*len(pipeids)
# Sets all created pipes to work as Check Valved to prevent backflow
status=['CV']*len(pipeids)
# sets all minor loss to 0
minorloss=[0]*len(pipeids)
# Assemble all lists into a dataframe where each row is the definition for one simple tank
added_pipes=pd.DataFrame(list(zip(pipeids,demand_nodes,reservoirids,lengths,diameters_pipes,hazen,minorloss,status,semicolons)))
# Exports the pipe section as a list of strings where each entry is a line of the pipes section
added_pipes=added_pipes.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Reservoir Coordinates
Each reservoir's coordinates are translated by a MM m in both x and y directions  
The coordinates of the tank nodes are merely for display and do not affect the simulation in any way  
[SCALE THESE VALUES FOR BETTER DISPLAY BASED ON YOUR INPUT FILE SCALE]

In [37]:
# Translates the tanks by a 100 m in both axes 
xcoordinates=[x+2 for x in xcoordinates]
ycoordinates=[y+2 for y in ycoordinates]

# Assemble all lists into a dataframe where each row is the coordinates for one simple tank
added_coordinates=pd.DataFrame(list(zip(reservoirids,xcoordinates,ycoordinates)))
# Exports the coordinate section as a list of strings where each entry is a line of the coordinates section
added_coordinates=added_coordinates.to_string(header=False,index=False,col_space=10).splitlines()

### Rewriting Base Demands
After creating the simple tanks, the base demands for all demand junctions has to be reset to zero

In [38]:
# List of zero base demands for all nodes
zerodemands=[0]*len(all_nodes)
# White space indicating no patterns
pattern=['     ']*len(all_nodes)
semicolons=[';']*len(all_nodes)
nodes=pd.DataFrame(list(zip(all_nodes,all_elevations,zerodemands,pattern,semicolons)))
nodes=nodes.to_string(header=False,index=False).splitlines()

### Writing the modified STM .inp File
We first read through the original .inp file to find the line positions of each of the sections  
Then each added or modified section is written into a new file and saved

In [39]:
# opens .inp file to read
file=open(path,'r')
lines=[]            # list to store all lines in the .inp file
linecount=0         # Counter for the number of lines
junctions_marker=0  # To store the line number at which the junctions section starts
reservoirs_marker=0 # To store the line number at which the reservoir section ends
pipes_marker=0      # To store the line number at which the pumps section starts
coords_marker=0     # To store the line number at which the vertices section starts

# Loops over each line in the input file 
for line in file:
    # Record the position of the phrase [JUNCTIONS] and add 2 to skip the header line
    if re.search('\[JUNCTIONS\]',line):
        junctions_marker=linecount+2
    # Record the position of the phrase [TANKS] and subtract 1 for the end of the reservoirs section
    if re.search('\[TANKS\]',line):
        reservoirs_marker=linecount-1
     # Record the position of the phrase [PUMPS] and subtract 1 to add pipes to the end of the pipe section
    if re.search('\[PUMPS\]',line):
        pipes_marker=linecount-1
     # Record the position of the phrase [Vertices] and subtract 1 to add Tank cooridnates to the end of the coordinates section
    if re.search('\[VERTICES\]',line):
        coords_marker=linecount-1
    linecount+=1
    # Store all lines in a list
    lines.append(line)
file.close()

# Translate the pipes marker by the length of the tank section that will be added before it (as it will displace all subsequent lines)
pipes_marker+=len(added_reservoirs)
# Translate the coordinates marker by the length of the added tanks and pipes
coords_marker+=len(added_reservoirs)+len(added_pipes)

# Inserts the created sections in their appropriate location in the list of lines
lines[junctions_marker:junctions_marker+len(nodes)]=nodes
lines[reservoirs_marker:reservoirs_marker]=added_reservoirs
lines[pipes_marker:pipes_marker]=added_pipes
lines[coords_marker:coords_marker]=added_coordinates

# Opens a new file in the same directory to write the modified network .inp file in
file=open(path.parent / (name_only+'_CV-Res.inp'),'w')
c=0     #line counter

# All lines added by this script are missing a new line character at the end, the conditional statements below add the new line character for these lines only and writes all lines to the file
for line in lines:
    if c>=junctions_marker and c<=junctions_marker+len(nodes):
        file.write(line+'\n')
    elif c>=reservoirs_marker and c<=reservoirs_marker+len(added_reservoirs):
        file.write(line+'\n')
    elif c>=pipes_marker and c<=pipes_marker+len(added_pipes):
        file.write(line+'\n')
    elif c>=coords_marker and c<=coords_marker+len(added_coordinates):
        file.write(line+'\n')
    else: file.write(line)    
    c+=1
file.close()


### References:
##### [1] S. Mohapatra, A. Sargaonkar, and P. K. Labhasetwar, “Distribution Network Assessment using EPANET for Intermittent and Continuous Water Supply,” Water Resources Management, vol. 28, no. 11, pp. 3745–3759, Sep. 2014, doi: 10.1007/s11269-014-0707-y.
##### [2] B. M. Janet Wagner, U. Shamir, and D. H. Marks, “WATER DISTRIBUTION RELIABILITY: SIMULATION METHODS.”